<a href="https://colab.research.google.com/github/plinder-org/plinder/blob/main/notebooks/plinder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Please run only the first cell, wait for the kernel to restart, and then continue with the second cell

In [ ]:
!pip install -q git+https://github.com/conda-incubator/condacolab.git@0.1.x
import condacolab
condacolab.install()

In [ ]:
!mamba install -q -c anaconda pip scipy networkx aivant::openstructure plip
!pip install -q networkit tabulate
!pip install -q pdb-validation@git+https://git.scicore.unibas.ch/schwede/ligand-validation.git
!pip install -q mmpdb@git+https://github.com/rdkit/mmpdb.git
!pip install -q plinder@git+https://github.com/plinder-org/plinder@revamp_loader
!pip install -q py3Dmol

# Downloading the dataset

The default location for the dataset is `~/.local/share/pinder/<PLINDER_RELEASE>/<PLINDER_ITERATION>` where:
- `<PLINDER_RELEASE>` is the date of the PDB sync used to generate the dataset (e.g 2024-06)
- `<PLINDER_ITERATION>` is the iteration of the source code that generated the dataset (e.g. `v2`)

If you want to use a different location, you can do so by setting the `PLINDER_MOUNT` environment variable. For example, in the Google Colab context we could set it to "/content" so that we can see the files in the file browser.


In [ ]:
%env PLINDER_LOG_LEVEL=0
%env PLINDER_MOUNT=/content


### Use `plinder_download` to download the complete dataset

```bash
plinder_download --help
usage:
    Download the full plinder dataset for the current configuration.
    Note that even though this is wrapped in a progress bar, the estimated
    completion time can vary wildly as it iterates over larger files vs.
    smaller ones.
    

optional arguments:
  -h, --help            show this help message and exit
  --release RELEASE     plinder release
  --iteration ITERATION
                        plinder iteration
  -y, --yes             skip confirmation
```

**NOTE: _This will take around an hour to complete and downloads around 1TB of data_**.

But after this is done you can set the `PLINDER_OFFLINE` environment variable to `true` to avoid downloading the data again.

### Use the `plinder` Python package to lazily access the dataset

Alternatively, if the `PLINDER_OFFLINE` environment variable is unset (which is the default), the dataset will be downloaded on lazily and on the fly as you access the data. This is preferred for exploration and prototyping as you don't need to download the entire dataset at once and can just work with the assets you need for your use-case.

In [ ]:
import plinder.core.utils.config

cfg = plinder.core.get_config()
print(f"local directory: {cfg.data.plinder_dir}")
print(f"remote data directory: {cfg.data.plinder_remote}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import py3Dmol

import tempfile
from biotite.structure.io import pdbx

# The PLINDEX


## Querying and filtering the index

Your main entry point to the dataset is the annotations table or the **index**, which is a Parquet file containing all the annotations for each system in the dataset.

**NOTE: _The entire file has 745 columns, >1.3M rows, and takes 24G of RAM to load into memory_**

We provide a `query_index` function to access the index and filter it based on the columns and criteria you need.

In [ ]:
from plinder.core.scores import query_index

The core of the PLINDER dataset is a collection of Protein-Ligand Interaction (PLI) systems extracted from the Protein Data Bank (PDB). The curation process in a nutshell is as follows:

1. For every PDB entry, we generate all available biological assemblies (biounits).
2. For each biounit, we identify all ligands and all protein chains within 6 Å of any ligand.
3. Ligands within 4 Å of each other are merged into a single PLI system.
4. For each system, we provide a range of detailed annotations and files to facilitate in-depth analysis and to enable a variety of use-cases.

Thus, a PLI **_system_** is defined as a collection of **_protein_** and **_ligand_** chains that are in close proximity to each other within a given **_biounit_** of a given PDB **_entry_**. The **_pocket_** of a system is defined as the set of protein residues within 6 Å of the ligands in the system.

Let's look at all the columns that define a `system`:

In [ ]:
plindex = query_index(
    columns=[
        "system_id",
        "entry_pdb_id",
        "system_biounit_id",
        "system_protein_chains_asym_id",
        "system_ligand_chains_asym_id",
        "ligand_instance_chain",
    ]
)
plindex.head(20)

Thus, a system is uniquely qualified by its `system_id` which is a combination of
- `entry_pdb_id` - the PDB identifier
- `system_biounit_id` - the biological assembly identifier
- `system_protein_chains_asym_id` - The list of protein chains in the system defined by their `<instance>.<label_asym_id>`
- `system_ligand_chains_asym_id` - The list of ligand chains in the system defined by their `<instance>.<label_asym_id>`

Systems containing multiple ligands (e.g `4grc__1__1.A__1.C_1.D`) span multiple rows, where each row represents a different ligand in the system (as seen in the `ligand_instance_chain` column).

In [ ]:
print(f"Number of ligands: {plindex.shape[0]}")
print(f"Number of systems: {plindex.system_id.nunique()}")
print(
    f"Number of biounits: {plindex[['entry_pdb_id', 'system_biounit_id']].drop_duplicates().shape[0]}"
)
print(f"Number of PDB IDs: {plindex.entry_pdb_id.nunique()}")

While `query_index` by default loads all the systems in the `train` and `val` splits, not every returned system may be useful for training your model.

In [ ]:
plindex = query_index(
    columns=[
        "system_id",
        "system_num_protein_chains",
        "system_num_ligand_chains",
        "system_type",
        "ligand_is_ion",
        "ligand_is_artifact",
        "ligand_is_cofactor",
        "ligand_is_fragment",
    ]
)

You may only be interested in systems with a certain number of protein or ligand chains, or you may want to exclude systems with ions or cofactors:

In [ ]:
plindex.system_num_protein_chains.value_counts()

In [ ]:
plindex.system_num_ligand_chains.value_counts()

In [ ]:
plindex.ligand_is_ion.value_counts()

In [ ]:
plindex.ligand_is_artifact.value_counts()

In [ ]:
plindex.ligand_is_fragment.value_counts()

In [ ]:
plindex.ligand_is_cofactor.value_counts()

### Single-ligand single-protein predictors

As an example, we consider the case of training single-ligand single-protein models. One way to filter training data is as follows:

In [ ]:
plindex_single = query_index(
    filters=[
        ("system_num_ligand_chains", "==", 1),
        ("system_num_protein_chains", "==", 1),
    ],
    splits=["train"],
)

In [ ]:
plindex_single.system_id.nunique()

However, as PLINDER also considers ions and crystallization artifacts as ligands if they are within 4 Å of a non-ion non-artifact ligand, there are also systems in PLINDER which only have one "proper" ligand. So, another strategy would be to only train on the proper ligands and ignore the ions and artifacts in the system.

In [ ]:
plindex_single_proper = query_index(
    filters=[
        ("system_proper_num_ligand_chains", "==", 1),
        ("system_num_protein_chains", "==", 1),
        ("ligand_is_proper", "==", True),  # filters out all other ligands in the system
    ],
    splits=["train"],
)

In [ ]:
plindex_single_proper.system_id.nunique()

This can provide up to 20% more data for training, however, the caveat is that some of the interactions made by artifacts or ions may influence the binding pose of the "proper" ligand.

One could also choose to include multi-ligand systems but only train with one ligand at a time, and the same for multi-protein. These choices are up to the user and we provide the annotations to enable such choices.

## Annotations

There are 745 columns in the full index. Of course, not every one will be useful for your use-case so we'll go through some common categories of them and some  use-cases requiring different annotations. You can get the full list of columns with descriptions as below, and these are also described in the [index docs](https://plinder-org.github.io/plinder/dataset.html#annotation-tables-index).

In [ ]:
from plinder.data.docs import get_all_column_descriptions

column_descriptions = get_all_column_descriptions()
column_descriptions

### Structure quality

A core principle of PLINDER is to be able to annotate which systems are of high enough experimental structure quality to be reliably used as the _ground truth_ for measuring model performance. As the quality of experimentally resolved structures [can vary significantly](https://doi.org/10.1107/S2059798322011901) and many crystal structures with ligands contain missing atoms or missing residues in the binding site, comparing prediction results to lower quality structures can incorrectly skew the perception of their performance.

![quality](https://www.plinder.sh/blog/figure2.png)


You can access all the crystal structure validation information extracted from PDB validation reports as well as crystal contact annotations by looking for columns starting with "entry_validation", "system_pocket_validation", "system_ligand_validation" etc.

In [ ]:
column_descriptions[column_descriptions["Name"].str.startswith("entry_validation")]

In [ ]:
column_descriptions[
    column_descriptions["Name"].str.startswith("system_pocket_validation")
]

For simplicity, we have "system_pass_validation_criteria" as a column that can be used to filter systems which pass our quality definitions:

```python
class QualityCriteria:
    max_entry_resolution: float = 3.5
    max_entry_r: float = 0.4
    max_entry_rfree: float = 0.45
    max_entry_r_minus_rfree: float = 0.05
    ligand_max_num_unresolved_heavy_atoms: int = 0
    ligand_max_alt_count: int = 1  # misnomer: this counts number of total configurations
    ligand_min_average_occupancy: float = 0.8
    ligand_min_average_rscc: float = 0.8
    ligand_max_average_rsr: float = 0.3
    ligand_max_percent_outliers_clashes: float = 0
    ligand_max_fraction_atoms_with_crystal_contacts: float = 0
    pocket_max_num_unresolved_heavy_atoms: int = 0
    pocket_max_alt_count: int = 1  # same as above
    pocket_min_average_occupancy: float = 0.8
    pocket_min_average_rscc: float = 0.8
    pocket_max_average_rsr: float = 0.3
    pocket_max_percent_outliers_clashes: int = 100
```

In [ ]:
plindex = query_index(
    columns=["system_id", "system_pass_validation_criteria"],
    splits=["train", "val", "test"],
)

In [ ]:
data = (
    plindex.drop_duplicates("system_id")
    .sort_values(by="system_pass_validation_criteria")
    .groupby(["split", "system_pass_validation_criteria"])
    .system_id.count()
    .unstack()
)
data_percentage = data.div(data.sum(axis=1), axis=0) * 100
ax = data_percentage.plot(
    kind="bar", stacked=True, figsize=(5, 3), color=["firebrick", "darkseagreen"]
)
ax.set_xticklabels(
    [label.get_text().upper() for label in ax.get_xticklabels()], rotation=0
)
ax.set_xlabel("")
ax.set_ylabel("% passing criteria")
ax.get_legend().remove()
for container, count_data in zip(ax.containers, data.values.T):
    ax.bar_label(container, labels=count_data, label_type="edge")

### Structure completeness

Related to structure quality, one aspect that is important to consider when using these structures in your training is **completeness**. While the inputs for prediction may be a protein sequence and a SMILES string, the protein-ligand complex structure that you get from the PDB may not have all the residues in the protein or all the atoms in the ligand resolved.

In [ ]:
column_descriptions[column_descriptions["Name"] == "ligand_num_unresolved_heavy_atoms"]

In [ ]:
column_descriptions[
    column_descriptions["Name"].str.contains("num_unresolved")
    # & ~column_descriptions["Name"].str.contains("validation")
]

In [ ]:
plindex = query_index(
    columns=[
        "system_id",
        "ligand_num_unresolved_heavy_atoms",
        "ligand_num_heavy_atoms",
        "system_protein_chains_num_unresolved_residues",
        "system_pocket_validation_num_unresolved_heavy_atoms",
        "system_protein_chains_total_length",
    ],
    splits=["train", "val", "test"],
)

Here we calculate the fraction of unresolved residues in the protein chains and the fraction of unresolved heavy atoms in the ligand, to see their distributions across the splits.

In [ ]:
plindex[
    "system_protein_chains_total_num_unresolved_residues"
] = plindex.system_protein_chains_num_unresolved_residues.map(sum)
plindex["system_protein_chains_fraction_unresolved_residues"] = (
    plindex.system_protein_chains_total_num_unresolved_residues
    / plindex.system_protein_chains_total_length
)
plindex["ligand_fraction_unresolved_heavy_atoms"] = (
    plindex.ligand_num_unresolved_heavy_atoms / plindex.ligand_num_heavy_atoms
)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(10, 3), sharex=True, sharey=True)
grouped_data = plindex.drop_duplicates("system_id").groupby("split")
split_colors = {
    "train": "#ff9999",
    "test": "#66b3ff",
    "val": "#7cc27c",
}
for i, (split, data) in enumerate(grouped_data):
    data.system_protein_chains_fraction_unresolved_residues.hist(
        ax=axes[i], density=True, color=split_colors[split]
    )
    axes[i].set_title(split.upper())
    if i == 0:
        axes[i].set_ylabel("Density")
fig.suptitle(
    "Fraction of Unresolved Protein Residues (SEQRES length - no. resolved residues)"
)
plt.tight_layout()

In [ ]:
print("Percentage of ligands with no unresolved heavy atoms:")
for split in ["train", "val", "test"]:
    print(
        f'{split.capitalize()}: {100 * (plindex[plindex["split"] == split].ligand_fraction_unresolved_heavy_atoms == 0).sum() / plindex[plindex["split"] == split].shape[0]:.2f}%'
    )

### Pocket domains

We annotated domains from different databases onto the protein chains of each system and then picked the one spanning the pocket residues of the system as the domain of the system pocket.

In [ ]:
pocket_domain_columns = column_descriptions[
    column_descriptions["Name"].str.startswith("system_pocket")
    & ~column_descriptions["Name"].str.startswith("system_pocket_validation")
]
pocket_domain_columns

In [ ]:
plindex = query_index(
    columns=["system_id"] + list(pocket_domain_columns.Name),
)

In [ ]:
plindex.drop_duplicates("system_id").system_pocket_ECOD_t_name.value_counts().head(10)

In [ ]:
plindex.drop_duplicates("system_id").system_pocket_kinase_name.value_counts().head(10)

### Ligand properties

Molecular properties and annotations are calculated from the ligand SMILES strings

In [ ]:
properties = [
    "ligand_molecular_weight",
    "ligand_num_rot_bonds",
    "ligand_num_rings",
    "ligand_num_hbd",
    "ligand_num_hba",
    "ligand_num_heavy_atoms",
    "ligand_crippen_clogp",
    "ligand_qed",
    "ligand_tpsa",
    "ligand_is_kinase_inhibitor",
]
column_descriptions[column_descriptions["Name"].isin(properties)]

In [ ]:
plindex = query_index(
    columns=["system_id", "ligand_instance_chain", "ligand_unique_ccd_code"]
    + properties,
    splits=["train", "val", "test"],
    filters=[
        ("ligand_is_proper", "==", True)  # focusing on non-ion, non-artifact ligands
    ],
)
plindex.head(20)

In [ ]:
binwidths = {
    "ligand_molecular_weight": ("Molecular weight", 50),
    "ligand_num_heavy_atoms": ("Number of heavy atoms", 5),
    "ligand_num_rot_bonds": ("Number of rotatable bonds", 2),
    "ligand_num_rings": ("Number of rings", 1),
    "ligand_tpsa": ("Topological polar surface area", 10),
    "ligand_crippen_clogp": ("Crippen logP", 1),
}
fig, axes = plt.subplots(3, 2, figsize=(10, 7))
axes = axes.flatten()
for i, prop in enumerate(binwidths):
    ax = axes[i]
    plindex.groupby("split")[prop].plot(
        kind="hist",
        density=True,
        alpha=0.5,
        histtype="stepfilled",
        legend=True,
        ax=ax,
        color=split_colors,
        bins=np.arange(
            min(plindex[prop]),
            max(plindex[prop]) + binwidths[prop][1],
            binwidths[prop][1],
        ),
    )
    ax.set_xlabel(binwidths[prop][0])
    ax.set_ylabel("Density")
plt.tight_layout()
plt.show()

These properties were further used to categorize the ligands into different types.

In [ ]:
ligand_types = [
    f"ligand_is_{x}"
    for x in [
        "lipinski",
        "covalent",
        "cofactor",
        "oligo",
        "ion",
        "fragment",
        "artifact",
    ]
]
column_descriptions[column_descriptions["Name"].isin(ligand_types)]

These categories were also aggregated to the system level for easier filtering:

In [ ]:
column_descriptions[
    column_descriptions["Name"].isin(
        [n.replace("ligand_is", "system_ligand_has") for n in ligand_types]
    )
]

In [ ]:
plindex = query_index(
    columns=["system_id", "ligand_unique_ccd_code"] + ligand_types,
    splits=["train", "val", "test"],
)

In [ ]:
labels = [c.replace("ligand_is_", "").capitalize() for c in ligand_types]
bar_colors = plt.cm.Pastel2.colors
split_names = ["train", "val", "test"]
fig, axes = plt.subplots(1, 3, figsize=(10, 3))
for i, split in enumerate(split_names):
    ax = axes[i]
    split_data = plindex[plindex["split"] == split]
    bars = ax.bar(
        labels,
        split_data[ligand_types].mean().mul(100).to_list(),
        width=1,
        color=bar_colors,
        edgecolor="black",
        label=split,
        linewidth=1,
    )
    ax.set_xticks(np.arange(len(labels)), labels, rotation=70)
    ax.set_ylim(0, 100)
    ax.set_title(split.upper())
    counts = split_data[ligand_types].sum().to_list()
    for bar, count in zip(bars, counts):
        ax.text(
            bar.get_x() + bar.get_width() / 2.0,
            bar.get_height() + 2,
            f"{count}",
            ha="center",
            va="bottom",
            rotation=70,
            fontsize=10,
        )

### PLI-specific properties

There are also some ligand properties that are specific to the interaction with the particular protein pocket present in the system. These include things like:
- The kinds of interactions the ligand may have with the protein, calculated using PLIP
- Experimental binding affinity, pulled from BindingDB when available
- Potential ligand-protein crystal contacts, defined as ligand-protein contacts below 5 Å which are not in the same asymmetric unit (symmetry mates) and not in the system biounit

In [ ]:
pli_specific = [
    "system_fraction_atoms_with_crystal_contacts",
    "system_num_crystal_contacted_residues",
    "ligand_binding_affinity",
    "system_has_binding_affinity",
    "ligand_interactions",
    "system_num_interactions",
]
column_descriptions[column_descriptions["Name"].isin(pli_specific)]

In [ ]:
plindex = query_index(
    columns=["system_id"] + pli_specific,
    splits=["train", "val", "test"],
)

You could filter out systems with crystal contacts:

In [ ]:
(
    plindex.drop_duplicates("system_id").system_fraction_atoms_with_crystal_contacts > 0
).sum()

If your model has an additional component for predicting binding affinity, you could see how much data is available for training and evaluation:

In [ ]:
plindex.drop_duplicates("system_id").groupby(
    "split"
).system_has_binding_affinity.value_counts()

In [ ]:
plindex["ligand_binding_affinity"].hist()
plt.xlabel(r"$pK_i$ or $pK_D$")
plt.ylabel("No. of system ligands")

You could even look deeper into the kinds of protein-ligand interactions present in your training data:

In [ ]:
plindex.groupby("split").system_num_interactions.describe()

In [ ]:
plindex["ligand_interactions"].values[0]

## Beyond the split (optional)

Here we mainly focused on systems in the train/val/test splits as the train and val splits are the ones that can be used for training models to either participate in the PLINDER leaderboard or compare your models' performance to methods in the leaderboard.

However, PLINDER itself contains all PLI systems in the PDB (except those containing only crystallization artifacts), and you may be interested in this dataset as a whole. This can be queried with the `splits=["*"]` option.

In [ ]:
plindex = query_index(
    columns=[
        "system_id",
        "entry_pdb_id",
        "system_num_protein_chains",
        "system_num_ligand_chains",
    ]
    + ligand_types,
    splits=["*"],
)
print("No. ligands:", len(plindex))
print("No. systems:", plindex.system_id.nunique())
print("No. PDB entries:", plindex.entry_pdb_id.nunique())

In [ ]:
fig, ax = plt.subplots(figsize=(5, 3))
bars = ax.bar(
    labels,
    plindex[ligand_types].mean().mul(100).to_list(),
    width=1,
    color=bar_colors,
    edgecolor="black",
    label=split,
    linewidth=1,
)
ax.set_xticks(np.arange(len(labels)), labels, rotation=70)
ax.set_ylim(0, 100)
ax.set_title(f"No. PLINDER ligands: {len(plindex)}")
counts = plindex[ligand_types].sum().to_list()
for bar, count in zip(bars, counts):
    ax.text(
        bar.get_x() + bar.get_width() / 2.0,
        bar.get_height() + 2,
        f"{count}",
        ha="center",
        va="bottom",
        rotation=70,
        fontsize=10,
    )

print(plindex.system_num_protein_chains.value_counts())
print(plindex.system_num_ligand_chains.value_counts())

When looking at distributions across the entire dataset, you'll notice that there are quite a lot of ion systems, as well as systems with more than 5 protein or ligand chains.

**NOTE: _While we provide annotations for all systems, those which are not in the train and val splits may not be used for training models to participate in the MLSB PLINDER leaderboard._**

## Similarity clusters

Similarity between two protein-ligand complexes can occur at various levels, including protein sequence, structural features, binding pocket characteristics, or ligand and interaction properties. We calculated a comprehensive set of similarity metrics to cover every combination of these dimensions — from identical systems, where the protein, pocket, interactions and ligand are the same, to systems that differ across all levels.

The entire set of similarity metrics is described [here](https://plinder-org.github.io/plinder/dataset.html#clusters-clusters), and they were calculated across all pairs of systems having a Foldseek or MMseqs alignment. We then used graph clustering to group systems into clusters based on their similarity with a specific **_metric_** and **_threshold_**. There are three types of clusters available: strongly connected graph components (**_strong component_**), weakly connected graph components (**_weak component_**), and communities detected using asynchronous label propagation (**_community_**).

We can load the similarity clusters assigned to each system for different metrics, thresholds, and clustering types.

In [ ]:
column_descriptions[
    column_descriptions["Name"].str.startswith("pocket_")
    & column_descriptions["Name"].str.contains("community")
]

In [ ]:
column_descriptions[
    column_descriptions["Name"].str.startswith("pocket_lddt")
    & column_descriptions["Name"].str.contains("component")
]

These cluster labels can be used in many ways with the annotations.

### Example: ATP-binding pockets (optional)

For example, maybe you are interested in seeing how many different kinds of ATP-binding pockets we have. First, let's find all the analogs of ATP. We can do this by finding the 95% ECFP4 tanimoto similarity component of ATP and then getting the CCD codes of all the ligands in the same cluster:

In [ ]:
ligand_cluster_column = "tanimoto_similarity_max__95__strong__component"
atp_ligand_cluster = query_index(
    columns=[
        ligand_cluster_column,
    ],
    filters=[
        ("ligand_unique_ccd_code", "==", "ATP"),
        ("system_num_ligand_chains", "==", 1),
    ],
)[ligand_cluster_column].values[0]

atp_analogs = query_index(
    columns=[
        "system_id",
        "entry_pdb_id",
        "ligand_unique_ccd_code",
        "ligand_rdkit_canonical_smiles",
    ],
    filters=[
        (ligand_cluster_column, "==", atp_ligand_cluster),
        ("system_num_ligand_chains", "==", 1),
    ],
).drop_duplicates("ligand_unique_ccd_code")
atp_analogs_code_set = set(atp_analogs.ligand_unique_ccd_code.unique())
atp_analogs_code_set

Let's see how they look

In [ ]:
import mols2grid

grid = mols2grid.MolGrid(atp_analogs, smiles_col="ligand_rdkit_canonical_smiles")
grid.display(subset=["ligand_unique_ccd_code", "img"])

Now we can get the pocket clusters of all systems containing ATP analogs. Here we're using `pocket_qcov__50__weak__component`, meaning a system within a cluster has some other system in that cluster with which it shares at least 50% of pocket residues when aligned.

In [ ]:
pocket_cluster_column = "pocket_qcov__50__weak__component"
plindex = query_index(
    columns=[
        "system_id",
        "entry_pdb_id",
        "entry_release_date",
        pocket_cluster_column,
        "system_pocket_ECOD_t_name",
    ],
    filters=[
        (
            "ligand_unique_ccd_code",
            "in",
            atp_analogs_code_set,
        ),  # only ATP-binding systems
        ("ligand_num_interactions", ">", 2),  # with >2 interactions with ATP
        ("system_num_ligand_chains", "==", 1),  # and ATP is the only ligand
    ],
)

In [ ]:
(
    "No. systems:",
    plindex.system_id.nunique(),
    "No. PDB entries:",
    plindex.entry_pdb_id.nunique(),
    "No. clusters:",
    plindex[pocket_cluster_column].nunique(),
)

In [ ]:
plindex[pocket_cluster_column].value_counts().head(10)

When we look at the ECOD topology names of the cluster "c11" for example, we see that these systems are GroEL equatorial domain-like domains. Interestingly, 7 systems from 2 different PDB entries don't have ECOD annotations yet as they were recently released, but are indeed [the same domain](https://doi.org/10.1038/s41467-024-45242-x).

In [ ]:
plindex[plindex[pocket_cluster_column] == "c11"].system_pocket_ECOD_t_name.value_counts(
    dropna=False
)

In [ ]:
plindex[
    (plindex[pocket_cluster_column] == "c11")
    & (plindex["system_pocket_ECOD_t_name"].isna())
].drop_duplicates("entry_pdb_id")

In [ ]:
plindex[plindex["system_pocket_ECOD_t_name"].isna()]["entry_pdb_id"].nunique()

### Example: Diversity sampling (optional)

The `pli_unique_qcov__50__community` column clusters systems such that the protein and ligand make similar interactions with the pocket. This clustering combines protein sequence and structural similarity (needed to obtain the pocket alignment), as well as the ligand-pocket interactions, making it a good proxy for unique kinds of binding present in the dataset.

Here's an example of how one might use `torch.utils.data.WeightedRandomSampler` to sample training systems based on their PLI community cluster.

In [ ]:
from torch.utils.data import WeightedRandomSampler

cluster_column = "pli_unique_qcov__50__community"

# Get train systems and their cluster labels
plindex = query_index(
    columns=["system_id", "entry_pdb_id", cluster_column], splits=["train"]
).drop_duplicates("system_id")

# Add the number of systems in each cluster to the dataframe
plindex = plindex.merge(
    plindex[cluster_column].value_counts().rename("cluster_num_systems"),
    left_on=cluster_column,
    right_index=True,
).reset_index(drop=True)

# Add the number of PDB entries in each cluster to the dataframe
plindex = plindex.merge(
    plindex.drop_duplicates("entry_pdb_id")[cluster_column]
    .value_counts()
    .rename("cluster_num_entries"),
    left_on=cluster_column,
    right_index=True,
).reset_index(drop=True)

# Ignore clusters with only one PDB entry
sample_from = plindex[plindex["cluster_num_entries"] > 1].reset_index(drop=True)

# Calculate the weight for each cluster
cluster_weights = 1.0 / sample_from.cluster_num_systems.values

# Create a WeightedRandomSampler and sample systems from the train set
sampler = WeightedRandomSampler(
    weights=cluster_weights, num_samples=len(cluster_weights)
)
sampled_indices = list(sampler)
sampled_plindex = (
    sample_from.iloc[sampled_indices]
    .reset_index(drop=True)
    .drop(columns=["cluster_num_systems", "cluster_num_entries"])
)

# Add the number of sampled systems in each cluster to the sampled dataframe
sampled_plindex = sampled_plindex.merge(
    sampled_plindex[cluster_column].value_counts().rename("cluster_num_systems"),
    left_on=cluster_column,
    right_index=True,
).reset_index(drop=True)

# Add the number of sampled PDB entries in each cluster to the sampled dataframe
sampled_plindex = sampled_plindex.merge(
    sampled_plindex.drop_duplicates("entry_pdb_id")[cluster_column]
    .value_counts()
    .rename("cluster_num_entries"),
    left_on=cluster_column,
    right_index=True,
)

In [ ]:
print("No. of original systems: ", sample_from.system_id.nunique())
print("No. of nonredundant sampled systems: ", sampled_plindex.system_id.nunique())
print("No. of original clusters: ", sample_from[cluster_column].nunique())
print("No. of sampled clusters: ", sampled_plindex[cluster_column].nunique())

Let's see how the sampling process has affected the distribution of cluster sizes.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 3))

cluster_sizes = (
    sample_from.drop_duplicates(cluster_column)
    .cluster_num_systems.value_counts()
    .sort_index()
)
ax[0].hist(cluster_sizes.index, weights=cluster_sizes.values, bins=20, log=True)
ax[0].set_xlabel("Cluster Size")
ax[0].set_ylabel("# Clusters (log scale)")
ax[0].grid(True, which="both", ls="-", alpha=0.2)

cluster_sizes = (
    sampled_plindex.drop_duplicates(cluster_column)
    .cluster_num_systems.value_counts()
    .sort_index()
)
ax[1].hist(cluster_sizes.index, weights=cluster_sizes.values, bins=20, log=False)
ax[1].set_xlabel("Cluster Size")
ax[1].set_ylabel("# Clusters")

## Beyond the PLINDEX (optional)

Not everything we annotated made it into the index file, as we tried to compromise between accessibility and usability. Here's how to get some more data from the additional files that are part of the dataset.

### Annotation JSON files

With `plinder.core.index.utils.load_entries` you can load the JSON file of a particular PDB ID and access all the information we extracted and saved for that ID.



In [ ]:
from plinder.core.index.utils import load_entries

In [ ]:
entries = load_entries(pdb_ids=["4agi"])

In [ ]:
entries["4agi"].keys()

In [ ]:
entries["4agi"]["systems"]["4agi__1__1.C__1.W"].keys()

In [ ]:
entries["4agi"]["systems"]["4agi__1__1.C__1.W"]["ligands"][0].keys()

### Annotation PyDantic classes

With `plinder.data.pipeline.utils.load_entries_from_zips` you can reconstruct the PyDantic dataclasses of the `Entry`, `System` and `Ligand`s to get access to more properties (though this requires a developer install of plinder with more dependencies)

In [ ]:
from plinder.data.pipeline.utils import load_entries_from_zips
from pathlib import Path

entries = load_entries_from_zips(data_dir=Path(cfg.data.plinder_dir), pdb_ids=["4agi"])

In [ ]:
import pandas as pd

In [ ]:
rows = []
for name, (description, ftype) in entries["4agi"].get_descriptions_and_types().items():
    rows.append((name, description, ftype))
pd.DataFrame(rows, columns=["Name", "Description", "Type"])

In [ ]:
entries["4agi"].validation

In [ ]:
rows = []
for name, (description, ftype) in (
    entries["4agi"].systems["4agi__1__1.C__1.W"].get_descriptions_and_types().items()
):
    rows.append((name, description, ftype))
pd.DataFrame(rows, columns=["Name", "Description", "Type"])

In [ ]:
rows = []
for name, (description, ftype) in (
    entries["4agi"]
    .systems["4agi__1__1.C__1.W"]
    .ligands[0]
    .get_descriptions_and_types()
    .items()
):
    rows.append((name, description, ftype))
pd.DataFrame(rows, columns=["Name", "Description", "Type"])

In [ ]:
entries["4agi"].systems["4agi__1__1.C__1.W"].pocket_residues

In [ ]:
entries["4agi"].systems["4agi__1__1.C__1.W"].ligands[0].interactions

In [ ]:
entries["4agi"].chains["C"].residues[22]

### Similarity scores



If you want more direct similarities between systems than the provided cluster labels, we also provide the entire set of calculated similarity scores, in three kinds of similarity datasets:
- Similarity between ligand bound systems (`holo`)
- Similarity between holo systems and unbound protein structures (`apo`) where "unbound" can also include only ion-bound and only artifact-bound.
- Similarity between holo systems and Alphafold predicted structures (`pred`) where the search was performed on all AFDB models having a UniProt ID present in any of the holo systems.

Any of these could be specified in `query_protein_similarity()`

**NOTE: _The "holo" search database takes 68G of disk space and, depending on the metric, threshold and filters used, can take up to a terabyte of RAM to query._**

The `apo` and `pred` search databases on the other hand are the result of a Foldseek + MMseqs search with a very strict coverage and identity threshold of 0.9. Thus, they are much smaller and lighter to query, and do not contain very sensitive or remote similarities. We'll see an example of querying these later on.

Here is an example of using the `query_protein_similarity` function to get the similarities > 50 between a "train" set and a "test" set for the `pli_unique_qcov` metric

```python
# Example train systems
train = ["7jxf__1__1.A_1.B__1.G", "1jtu__1__1.A_1.B__1.C_1.D",
         "8f9d__2__1.C_1.D__1.G", "6a9a__1__1.A_2.A__2.C_2.D",
         "1b5e__2__1.A_1.B__1.D"]
# Example test systems
test = ["1b5d__1__1.A_1.B__1.D", "1s2g__1__1.A_2.C__1.D",
       "4agi__1__1.C__1.W", "4n7m__1__1.A_1.B__1.C",
         "7eek__1__1.A__1.I"]

metric = "pli_unique_qcov"
threshold = 50
query_protein_similarity(
        search_db="holo",
        columns=["query_system", "target_system", "similarity"],
        filters=[
                ("query_system", "in", test),
                ("target_system", "in", train),
                ("metric", "==", metric),
                ("similarity", ">=", threshold),
            ],
)
```

For ligand similarities, one can use the following approach:

In [ ]:
from plinder.core.scores import query_index

plindex = query_index(
    columns=["system_id", "ligand_rdkit_canonical_smiles"], splits=["val"]
)

In [ ]:
# Example test and train

test = set(plindex.system_id.values[:10])
train = set(plindex.system_id.values[20:50])

In [ ]:
from plinder.data import smallmolecules

plindex["fp"] = plindex["ligand_rdkit_canonical_smiles"].map(
    smallmolecules.mol2morgan_fp
)
test_set = plindex.loc[plindex["system_id"].isin(test)].reset_index(drop=True)
test_set["tanimoto_similarity_max"] = smallmolecules.tanimoto_maxsim_matrix(
    plindex.loc[plindex["system_id"].isin(train)]["fp"].to_list(),
    test_set["fp"].to_list(),
)

In [ ]:
test_set["tanimoto_similarity_max"]

# Accessing system files



We provide files for all holo PLINDER systems with <6 protein and <6 ligand chains. These can be accessed with the `PlinderSystem` object which also does the work of downloading and extracting only the relevant files.

In [ ]:
from plinder.core import PlinderSystem

plinder_system = PlinderSystem(system_id="4agi__1__1.C__1.W")

We can visualise the system using py3Dmol

In [ ]:
view = py3Dmol.view()
view.setBackgroundColor("white")

view.addModel(open(plinder_system.system_cif, "r").read(), "cif")
view.setStyle({"chain": "1.C"}, {"cartoon": {"color": "purple"}})
view.setStyle({"chain": "1.W"}, {"stick": {"colorScheme": "elem"}})

view.zoomTo()
view.show()

### Ligand

The ligands are provided in SDF format in `ligand_sdfs`.


In [ ]:
plinder_system.ligand_sdfs

And the corresponding SMILES strings in `smiles`.

In [ ]:
plinder_system.smiles

### Receptor

The CIF/PDB files of the receptor are stored in `receptor_cif` and `receptor_pdb` and only contain the protein chains of the system.

In [ ]:
plinder_system.receptor_pdb, plinder_system.receptor_cif

We recommend using the CIF file as PDB is an obsoleted format. However, if you must use the PDB file, an additional consideration is that the chains are renamed to single letters, which you can access with the `chain_mapping` attribute.


In [ ]:
plinder_system.chain_mapping

The FASTA file and sequences of the receptor are stored in `sequences_fasta` and `sequences` respectively. These are the canonical sequences of all protein chains in the system.


In [ ]:
plinder_system.sequences_fasta, plinder_system.sequences

### Linked structures

Where possible, we've linked plinder systems to associated apo structures from the PDB and predicted structures from AFDB. This was done using the same kind of similarity searches used for system clustering except with strict restrictions on the sequence identity and coverage of linked structures.

The `linked_structures` attribute is a pandas DataFrame with information on the links for a system which were both found and additionally scored for conformational difficulty. This additional scoring consists of superposing the found apo or predicted chain to the receptor of the system with global sequence-based alignment, transplanting the ligand to the found structure, and evaluating the resulting protein-ligand complex as though it were a predicted structure for the given system. So, the `linked_structures` DataFrame contains the similarity scores from the alignments as well as the metrics from the evaluation.

In [ ]:
link_info = plinder_system.linked_structures

In [ ]:
link_info[
    [
        "id",
        "pocket_fident",
        "lddt",
        "bb_lddt",
        "lddt_lp_ave",
        "lddt_pli_ave",
        "bisy_rmsd_ave",
        "sort_score",
        "kind",
    ]
]

For example, here we can see that "4uou_B"
- has 100% identical residues corresponding to the pocket of the system
- has a very high lDDT and backbone lDDT scores, indicating that the structure is very similar to the receptor.
- has a `sort_score` of 2.4, which is the resolution for an apo structure and the plDDT score for a predicted structure.

Indeed the superposition + transplant results show the same story
- a global superposition puts the ligand in the right place (seen by the `bisy_rmsd` of the ligand pose),
- the distances between the pocket atoms are similar (seen by the `lddt_lp_ave` metric),
- and the distances between the ligand and protein atoms are similar (seen by the `lddt_pli_ave` metric).

`get_linked_structure` then gives the file path to the found structure

NOTE: if you have not previously downloaded the dataset, the below command may take a couple of minutes to downloaded the relevant files

In [ ]:
plinder_system.get_linked_structure("apo", "4uou_B")

### Linking more structures (optional)

We attempted to link all single protein chain holo systems to up to 5 apo structures and up to 5 predicted structures and additionally evaluated these links for conformational difficulty.

All of these can be accessed with `query_links`

In [ ]:
from plinder.core.scores import query_index, query_links

In [ ]:
plindex = query_index(
    filters=[
        ("system_num_protein_chains", "==", 1),
    ]
)

In [ ]:
all_scored_links = query_links(
    filters=[("reference_system_id", "in", set(plindex.system_id))]
)
(
    len(all_scored_links),
    all_scored_links.reference_system_id.nunique(),
    all_scored_links.id.nunique(),
)

In [ ]:
plindex["has_scored_linked_structure"] = plindex.system_id.isin(
    all_scored_links.reference_system_id
)

In [ ]:
plindex.drop_duplicates("system_id").groupby(
    "split"
).has_scored_linked_structure.value_counts()

This is not complete coverage across the entire training set however, and has a number of caveats:
- _apo_ structures could not be found for all systems
- AFDB does not contain predicted structures for viral proteins
- We did not attempt to link multi-protein holo systems
- We only scored up to 5 links per system for performance reasons, sorting by resolution for apo and plDDT for predicted links
- Not all links have 100% identical pocket residues as we used a threshold of 95%
- Coverage was only checked for the system receptor (>80%) meaning the linked structure may be much longer


You may want to have more links per system, more 100% identical pocket links, or additionally have cross-docking structures as links. You can query the `apo`, `pred` (and `holo` but don't run that in Colab!) search scores directly to get such results, using the `multi_query_protein_similarity` function, which returns hits matching all given criteria:

In [ ]:
from plinder.core.scores.protein import multi_query_protein_similarity

In [ ]:
filter_criteria: dict[str, int] = {
    "pocket_fident": 100,  # 100% identical pocket residues
    "protein_fident_weighted_sum": 95,  # >=95% sequence identity of aligning region
    "protein_fident_qcov_weighted_sum": 80,  # >=80% sequence identity * coverage across full system receptor
    "pocket_lddt": 20,  # >=20 lDDT for pocket
}

multi_query_protein_similarity(
    system_id="4agi__1__1.C__1.W", search_db="apo", filter_criteria=filter_criteria
)

In [ ]:
multi_query_protein_similarity(
    system_id="4agi__1__1.C__1.W", search_db="pred", filter_criteria=filter_criteria
)

And get the file path to the found structure as before:

In [ ]:
plinder_system.get_linked_structure("pred", "Q4WW81_A")

Note that the `holo` search requires the holo scores dataset which is 68GB and thus will not be demonstrated in the notebook. However, you can use the same interface to query it and get the file path to the found structure in exactly the same way. An additional consideration for `holo` search is to restrict the search to the same split as the system being queried, the `splits` parameter in the `multi_query_protein_similarity` function controls this and is set to ["train"] by default.

### Augmenting your dataset with ESMFold structures (optional)


Linking can also be done with structure prediction. For example, here's how one could use ESMFold to predict a structure for the system receptor:

In [ ]:
from pathlib import Path

import requests
from biotite.structure.io.pdb import PDBFile, get_structure
from biotite.structure.io.pdbx import CIFFile, set_structure


def get_esmfold_prediction(sequence: str, filename: Path) -> str | None:
    url = "https://api.esmatlas.com/foldSequence/v1/pdb/"

    try:
        response = requests.post(url, data=sequence)
        response.raise_for_status()
        pdb_text = response.text
        with open(filename, "w") as f:
            f.write(pdb_text)
        structure = get_structure(
            PDBFile.read(filename),
            model=1,
        )
        write_file = CIFFile()
        set_structure(write_file, structure)
        write_file.write(filename.as_posix())
        return filename.as_posix()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

In [ ]:
filename = "esmfold_example.cif"
get_esmfold_prediction(plinder_system.sequences["1.C"], Path(filename))

In [ ]:
view = py3Dmol.view()
view.setBackgroundColor("white")

view.addModel(open("esmfold_example.cif", "r").read(), "cif")
view.setStyle({"chain": "A"}, {"cartoon": {"color": "blue"}})

view.addModel(open(plinder_system.system_cif, "r").read(), "cif")
view.setStyle({"chain": "1.C"}, {"cartoon": {"color": "purple"}})
view.setStyle({"chain": "1.W"}, {"stick": {"colorScheme": "elem"}})

view.zoomTo()
view.show()

**NOTE: _Please note the rules for augmentation when training a model to participate in the leaderboard_**

> If starting structures/conformations need to be generated for the model, then this can only be done from the training and validation sequences and SMILES. Note that this is only the case for train & validation - no external folding methods or starting structures are allowed for the test set under any circumstance! Only the predicted structures/conformers themselves may be used in this way, the embeddings or models used to generate such predictions may not. E.g. it is not valid to “distill” a method that was not trained on PLINDER.

# Aligning, masking, and featurizing

To enable using the system and linked structure files in training deep learning models, we've implemented a number of useful functions to align, mask, and featurize proteins and ligands.

For this, we convert our system to a `Structure` object.

In [ ]:
system_structure = plinder_system.holo_structure

## Ligand

In [ ]:
for name in system_structure.get_properties():
    if "ligand" in name:
        print(name)

The ligands are provided using dictionaries.

These dictionaries contain information for each ligand:
- `input_ligand_templates`: 2D RDKit mols generated from the RDKit canonical SMILES
- `input_ligand_conformers`: 3D (random) conformers generated for each input mol
- `input_ligand_conformers_coords`: positional coordintates for 3D conformers
- `resolved_ligand_mols`: RDKit mols of solved (holo) ligand structures
- `resolved_ligand_mols_coords`: positional coordintates for holo ligand structures
- `ligand_template2resolved_atom_order_stacks`: paired stacked arrays (template vs holo) mapping atom order by index
- `ligand_chain_ordered`: ordered list of all ligands by their keys

### Ligand atom id mapping mapping

Unlike the protein sequence - there is no canonical order to ligand atoms in the molecule.
It can be further complicated by automorphisms present in the structure due to symmetry, i.e. there is more than one match that is possible between the structures.

This is important when calculating ligand structure loss, as the most optimal atom order can change between the different inference results. Typically, it is accepted to take the atom ordering resulting in the best objective score and use that for the loss calculation.

Occasionally futher ambiguity arises to to part of the ligand structure being unresolved in the holo structure - this can lead to multiple available matches. We use RascalMCES algorithm from RDKit to provide all the possible matches between the atom order in the input structure (from SMILES) to the resolved holo structure.

This is provided as stacks of atom order arrays that reorder the template and holo indices to provide matches. Each stack is a unique order transformation and should be iterated.

In [ ]:
from rdkit.Chem.Draw import IPythonConsole

IPythonConsole.drawOptions.addAtomIndices = True
# IPythonConsole.ipython_3d = False

In [ ]:
system_structure.input_ligand_templates[system_structure.ligand_chain_ordered[0]]

In [ ]:
system_structure.input_ligand_conformers[system_structure.ligand_chain_ordered[0]]

In [ ]:
system_structure.resolved_ligand_mols[system_structure.ligand_chain_ordered[0]]

### Ligand conformer coordinates

As you can tell, the input 2D and 3D conformer indices match, but the resolved ligand is different.
Thus to perform a correct comparison for their coordinates one should use atom order stacks.


In [ ]:
(
    input_atom_order_stack,
    holo_atom_order_stack,
) = system_structure.ligand_template2resolved_atom_order_stacks[
    system_structure.ligand_chain_ordered[0]
]
input_atom_order_stack, holo_atom_order_stack

In [ ]:
system_structure.input_ligand_conformers_coords[
    system_structure.ligand_chain_ordered[0]
][input_atom_order_stack]

In [ ]:
system_structure.resolved_ligand_mols_coords[system_structure.ligand_chain_ordered[0]][
    holo_atom_order_stack
]

## Protein

In [ ]:
for name in system_structure.get_properties():
    if "protein" in name:
        print(name)

### Masking
The properties `protein_backbone_mask` and `protein_calpha_mask` are boolean masks that can be used to select backbone or calpha atoms from biotite `AtomArray`. The indices of `True` corresponds to backbone or calpha indices.

In [ ]:
print(
    "Total number of atoms:",
    len(system_structure.protein_atom_array),
)
print("Number of backbone atoms:", system_structure.protein_backbone_mask.sum())
print(
    "Number of calpha atoms:",
    system_structure.protein_calpha_mask.sum(),
)

In [ ]:
calpha_atom_array = system_structure.protein_atom_array[
    system_structure.protein_calpha_mask
]
calpha_atom_array.coord.shape

You can also filter by arbitrary properties of the `AtomArray` using the `filter` method. This returns a new `Structure` object.

In [ ]:
calpha_structure = system_structure.filter(
    property="atom_name",
    mask="CA",
)

In [ ]:
calpha_structure.protein_atom_array.coord.shape

## Linked protein input structures

For realistic inference scenarios we need to initialize our protein structures using a linked structure (introduced above). In most cases, these will not be a perfect match to the _holo_ structure - the number of residues, residue numbering, and sometime the sequnce can be different. It's important to be able to match these structures to ensure that we can map between them. In the example below, we will take a holo structure and it's linked predicted (pred) form with different number of residues and try to match and crop the resulting structures.

In [ ]:
plinder_system = PlinderSystem(system_id="4cj6__1__1.A__1.B")
holo = plinder_system.holo_structure
predicted = plinder_system.alternate_structures["P12271_A"]
print(
    "Number of residues in holo: ",
    holo.protein_atom_array.shape[0],
    "\n" "Number of residues in predicted: ",
    predicted.protein_atom_array.shape[0],
)

In [ ]:
view = py3Dmol.view()
view.setBackgroundColor("white")

view.addModel(open(plinder_system.system_cif).read(), "cif")
view.setStyle({"chain": "1.A"}, {"cartoon": {"color": "blue"}})
view.setStyle({"chain": "1.B"}, {"stick": {"colorScheme": "elem"}})

cif_file = pdbx.CIFFile()
pdbx.set_structure(cif_file, predicted.protein_atom_array)
with tempfile.NamedTemporaryFile() as tmp:
    cif_file.write(tmp.name)
    view.addModel(tmp.read().decode(), "cif")
    view.setStyle({"model": -1}, {"cartoon": {"color": "red"}})
view.zoomTo()
view.show()

Now, let's align the holo and predicted structures to figure out the correspondence between their residues. For this we use the `align_common_sequence` function of the holo `Structure` object, which aligns two structures based on their shared sequences. It has the following parameters:

```
other: Structure
    The other structure to align to
copy: bool
    Whether to make a copy or edit in-place
remove_differing_atoms: bool
    Whether to remove differing atoms between the two structure
renumber_residues: bool [False]
    If True, renumber residues in the two structures to match and starting from 1.
    If False, sets the resulting residue indices to the one from the aligned sequence
remove_differing_annotations: bool [False]
    Whether to remove differing annotations, like b-factor, etc
```
In this example, we will match, make copies and crop the structures.

**NOTE**: To use this function the proteins to be aligned must have the same chain ids. So, we first set the chain id of the predicted structure to that of the holo structure.

In [ ]:
predicted.set_chain(holo.protein_chain_ordered[0])
holo_cropped, predicted_cropped = holo.align_common_sequence(predicted)

In [ ]:
predicted_cropped_superposed, raw_rmsd, refined_rmsd = \
  predicted_cropped.superimpose(
    holo_cropped
)
print(raw_rmsd, refined_rmsd)

In [ ]:
view = py3Dmol.view()
view.setBackgroundColor("white")

cif_file = pdbx.CIFFile()
pdbx.set_structure(cif_file, holo_cropped.protein_atom_array)
with tempfile.NamedTemporaryFile() as tmp:
    cif_file.write(tmp.name)
    view.addModel(tmp.read().decode(), "cif")
    view.setStyle({"model": -1}, {"cartoon": {"color": "blue"}})

cif_file = pdbx.CIFFile()
pdbx.set_structure(cif_file, predicted_cropped_superposed.protein_atom_array)
with tempfile.NamedTemporaryFile() as tmp:
    cif_file.write(tmp.name)
    view.addModel(tmp.read().decode(), "cif")
    view.setStyle({"model": -1}, {"cartoon": {"color": "red"}})
view.zoomTo()
view.show()

We can see that the wiggly disordered region on the predicted protein structure has been cropped out.

In [ ]:
print(
    "Number of residues in holo: ",
    holo_cropped.protein_atom_array.shape[0],
    "\n" "Number of residues in predicted: ",
    predicted_cropped.protein_atom_array.shape[0],
)

# Datasets and Dataloaders

#### Interacting with the PLINDER dataset
`PlinderDataset` provides an interface to interact with _PLINDER_ data as a dataset. It is a subclass of `torch.utils.data.Dataset`, as such subclassing it and extending should be familiar to most users. Flexibility and general applicability is our top concern when designing this interface and `PlinderDataset` allows users to not only define their own split but to also bring their own featurizer.
It can be initialized with the following parameters
```
Parameters
    ----------
    df : pd.DataFrame | None
        the split to use
    split : str
        the split to sample from
    split_parquet_path : str | Path, default=None
        split parquet file
    input_structure_priority : str, default="apo"
        Which alternate structure to proritize
    featurizer: Callable[
            [Structure, int], dict[str, torch.Tensor]
    ] = structure_featurizer,
        Transformation to turn structure to input tensors
    padding_value : int
        Value for padding uneven array
    **kwargs : Any
        Any other keyword args
```

For an example of how to write your own featurizer see [Featurizer Example](https://github.com/plinder-org/plinder/blob/c36eef9b02823ce572de905c094f6c85c03800ca/src/plinder/core/loader/featurizer.py#L16). The signature is shown below:
```
def structure_featurizer(
    structure: Structure, pad_value: int = -100
    ) -> dict[str, Any]:
```
The input is a `Structure` object and it returns dictionary of padded tensor features.

In [ ]:
from plinder.core.loader.dataset import PlinderDataset, get_torch_loader

This is where you may want to load a `train` dataset, but for the purposes of demonstration - we will start with `val` due to smaller memory footprint, and load only single protein chain systems.

In [ ]:
val_dataset = PlinderDataset(
    split="val", filters=[("system_num_protein_chains", "==", 1)]
)

In [ ]:
val_dataset[2]

In [ ]:
val_loader = get_torch_loader(val_dataset)

In [ ]:
for data in val_loader:
    test_torch = data
    break

# Evaluation

## `plinder_eval`

We provide a wrapper around the OpenStructure scorer as well as PoseBusters to make it easy to evaluate predictions against PLINDER systems and generate scores for the leaderboard.


This can be accessed through the command-line tool `plinder_eval`





In [ ]:
!plinder_eval --help

This expects a CSV file with each row representating a protein-ligand pose, and the following columns:

- `id`: An identifier for the prediction (same across different ranked poses of the same prediction)
- `reference_system_id`: `plinder` system ID to use as reference
- `receptor_file`: Path to predicted receptor CIF file. Leave blank if rigid docking, the system's receptor file will be used.
- `rank`: The rank of the predicted pose (1-indexed)
- `ligand_file`: Path to the SDF file of the predicted pose

And produces a `scores.parquet` file with evaluated metrics for all the poses. For individual systems you can also use the Python function `plinder.eval.docking.write_scores.evaluate`.

### Example: vina
For example, here's how one could use AutoDock Vina to generate some docked poses from one of the apo/pred structures and then evaluate them:

In [ ]:
!mamba install -q vina

In [ ]:
from biotite.structure import centroid, from_template
import numpy as np
from biotite.application.autodock import VinaApp
from biotite.structure.io import load_structure
from biotite.structure.io.mol import MOLFiles
from pathlib import Path


def vina(
    ligand, receptor, pocket_center, output_folder: Path, size=10, max_num_poses=5
):
    app = VinaApp(
        ligand,
        receptor,
        center=pocket_center,
        size=[size, size, size],
    )
    app.set_max_number_of_models(max_num_poses)
    app.start()
    app.join()
    docked_ligand = from_template(ligand, app.get_ligand_coord())
    docked_ligand = docked_ligand[..., ~np.isnan(docked_ligand.coord[0]).any(axis=-1)]
    output_files = []
    for i in range(docked_ligand.shape[0]):
        sdf_file = MOLFile()
        sdf_file.set_structure(docked_ligand[i])
        output_file = output_folder / f"docked_ligand_{i}.sdf"
        sdf_file.write(output_file)
        output_files.append(output_file)
    return output_files

In [ ]:
plinder_system = PlinderSystem(system_id="4cj6__1__1.A__1.B")
apo_structure = plinder_system.alternate_structures["P12271_A"]
input_protein = apo_structure.protein_atom_array
input_ligand = plinder_system.holo_structure.input_ligand_conformer_atom_array["1.B"]

output_folder = Path("vina_output")
output_folder.mkdir(exist_ok=True)
docking_poses = vina(
    ligand=input_ligand,
    receptor=input_protein,
    pocket_center=centroid(input_protein),
    output_folder=output_folder,
    max_num_poses=5,
    size=20,
)

In [ ]:
from plinder.eval.docking.write_scores import evaluate

metrics = []
for pose in docking_poses:
    metrics.append(
        evaluate(
            model_system_id=apo_structure.id,
            reference_system_id=plinder_system.system_id,
            receptor_file=apo_structure.protein_path,
            ligand_files=[pose],
            flexible=False,
            posebusters=False,
            posebusters_full=False,
        )
    )

In [ ]:
import pandas as pd

metrics_df = pd.DataFrame(metrics)
metrics_df[["reference", "lddt_pli_ave", "lddt_lp_ave", "bisy_rmsd_ave"]]

In [ ]:
metrics_full = []
for pose in docking_poses:
    metrics_full.append(
        evaluate(
            model_system_id=apo_structure.id,
            reference_system_id=plinder_system.system_id,
            receptor_file=apo_structure.protein_path,
            ligand_files=[pose],
            flexible=True,
            posebusters=True,
            posebusters_full=False,
        )
    )

In [ ]:
metrics_df = pd.DataFrame(metrics_full)
metrics_df[
    [
        "reference",
        "lddt",
        "bb_lddt",
        "per_chain_lddt_ave",
        "per_chain_bb_lddt_ave",
        "lddt_pli_ave",
        "lddt_lp_ave",
        "bisy_rmsd_ave",
    ]
]

In [ ]:
metrics_df[[c for c in metrics_df.columns if c.startswith("posebusters")]]

In [ ]:
view = py3Dmol.view()
view.setBackgroundColor("white")
view.setStyle({"stick": {"color": "green"}})
view.addModel(open(apo_structure.protein_path, "r").read(), "cif")
view.setStyle({"chain": "A"}, {"cartoon": {"color": "blue"}})
view.addModel(open(docking_poses[-1], "r").read(), "sdf")
view.setStyle({"model": -1}, {"stick": {"colorScheme": "elem"}})

In [ ]:
view = py3Dmol.view()
view.setBackgroundColor("white")
view.addModel(open(plinder_system.system_cif, "r").read(), "cif")
view.setStyle(
    {"chain": plinder_system.holo_structure.protein_chain_ordered[0]},
    {"cartoon": {"color": "purple"}},
)
view.setStyle(
    {"chain": plinder_system.holo_structure.ligand_chain_ordered[0]},
    {"stick": {"color": "green"}},
)
view.zoomTo()
view.show()

## `plinder_stratify`

(This command will not need to be run by a user, the `test_set.parquet` and `val_set.parquet` file will be provided with the split release)

```bash
plinder_stratify --split_file split.parquet --output_dir results --train_label train --test_label val
```

Makes `val_set.parquet` which

- Labels the maximum similarity of each val system to the training set across all the similarity metrics
- Stratifies the val set based on training set similarity into `novel_pocket_pli`, `novel_ligand`, `novel_protein`, `novel_all` and `not_novel`
- Labels val systems with high quality.


In [ ]:
val_stratification = pd.read_parquet(
    Path(cfg.data.plinder_dir)
    / "splits"
    / "stratification"
    / "train_vs_val_data"
    / "val_set.parquet"
)
val_stratification.head()

In [ ]:
val_stratification[[c for c in val_stratification.columns if "novel" in c]].sum()

## `plinder_plot`

```bash
plinder_plot --scores_file scores.parquet --data_file val_set.parquet --output_dir results
```

Plots performance of a model stratified by training set similarity.

![plot](https://www.biorxiv.org/content/biorxiv/early/2024/07/19/2024.07.17.603955.1/F5.large.jpg)